# GWO hourly meteorological data plotting and processing
**Author: Jun Sasaki  Coded on 2019-01-10  Updated on 2024-07-02**<br>
Handling of Japan Meteorological Agency [GWO](http://www.roy.hi-ho.ne.jp/ssai/mito_gis/).
- For details, see **mod_class_met.py**, which was developed in **dev_class_met.ipynb**.

In [ ]:
import os
from pathlib import Path
DATA_DIR = Path(os.environ.get("DATA_DIR", "/mnt/c/Data")).expanduser()
GWO_HOURLY_DIR = DATA_DIR / "met" / "JMA_DataBase" / "GWO" / "Hourly"

In [ ]:

from gwo_amd.mod_class_met import *
from datetime import datetime
dirpath = str(GWO_HOURLY_DIR)
#dirpath = "c:/dat/met/GWO/Hourly"


In [ ]:
datetime_ini = "2019-12-1 00:00:00"
datetime_end = "2020-12-30 00:00:00"
#datetime_ini = "2010-9-2 00:00:00"
#datetime_end = "2010-9-3 00:00:00"
stn = "Chiba"
met = Met_GWO(datetime_ini=datetime_ini, datetime_end=datetime_end, stn=stn, dirpath=dirpath)

## Gets pandas DataFrame
pandas DataFrame can be obtained by `met.df`

In [ ]:
met.df.head()

In [ ]:
met.df.columns

In [ ]:
met.df['kion'].head()

# Plotting example using [Matplotlib](https://matplotlib.org/stable/index.html)

In [ ]:
data = Data1D(df=met.df, col_1='kion')

In [ ]:
data.v1

## Example of scalar 1-D time series plot
- Rolling mean is applied to `Plot1D()` by setting its arguments of `window` in odd integer number and `center` (default is `True`).

In [ ]:
### xlim = (parse("2014-01-15"), parse("2014-02-16"))  ### ex. for datetime
ylabel='Temperature (degC)'
xlim = None
#xlim = (parse("1990-09-02"), parse("1992-09-03"))
dx = 7
ylim = None
dy = 2

## Set window=1 when no plot.
window=1
try:
    plot_config = Data1D_PlotConfig(xlim = xlim, ylim = ylim, x_minor_locator = DayLocator(interval=dx),
                                    y_minor_locator = MultipleLocator(dy),
                                    format_xdata = DateFormatter('%Y-%m-%d'), ylabel = ylabel)
    Plot1D(plot_config, data, window=window, center=True).save_plot('data.png', dpi=600)
except:
    print('dx and/or dy may be too small and too many tickes.')

## Example of time series wind vector plot with its speed

In [ ]:
wind = Data1D(met.df, 'u', 'v')
print(wind.v[0:10])

In [ ]:
#xlim = (parse("2013-12-25 00:00:00"),parse("2014-01-10 00:00:00"))
vlabel = 'Wind speed (m/s)'
ylabel = 'Wind vector (m/s)'
png_vector = "hourly_wind.png"
xlim = None
#xlim = (parse("1990-09-02"), parse("1990-09-03"))
dx = 7
#ylim = None
ylim = (-15, 15)
dy = 1

## Set window=1 when no plot. 
window=1
try:
    plot_config = Data1D_PlotConfig(xlim = xlim, ylim = ylim, x_minor_locator = DayLocator(interval=dx),
                                    y_minor_locator = MultipleLocator(dy), format_xdata = DateFormatter('%Y-%m-%d'),
                                    ylabel=ylabel, vlabel=vlabel, vlabel_loc = 'lower center')
    Plot1D(plot_config, wind, window=window, center=True).save_vector_plot(png_vector, magnitude = True, dpi=600)
except:
    print('dx and/or dy may be too small and too many ticks')

# Plotting exmaples using [hvPlot](https://hvplot.holoviz.org/)
This is for interactive plotting but not suitable for saving graphics into files.

In [ ]:
import hvplot.pandas

In [ ]:
data.df[['kion', 'sped']].hvplot()

In [ ]:
def hook(plot, element):
    plot.handles['xaxis'].axis_label_text_font_style = 'normal'
    plot.handles['yaxis'].axis_label_text_font_style = 'normal'
    

### How to specify options for hvPlot (オプション指定方法)
- Many of the Holoviews options can be specified with the hvPlot argument.
- `hooks` defines the function `hook(plot, element)`, which is specified in hvPlot as `.opts(hooks=[hook])`. Unconfirmed, but it seems that all options can be specified, including Bokeh options that are not defined as arguments in hvPlot or Holoviews.
- 基本的にはhvplotの引数でHoloviewsのオプションの多くが指定できる
- `hooks`は関数`hook(plot, element)`を定義し，hvPlotで`.opts(hooks=[hook])`として指定する．未確認だが，hvPlotやHoloviewsの引数としては定義されていないBokehのオプションを含め，すべてのオプションが指定できそう

In [ ]:
data.df['kion'].hvplot(xlim=(datetime(2020,1,2), datetime(2020,3,4)), xticks=10,
                       ylabel='Temperature (degC)', ylim=(-4,35), yticks=10,
                       width=600, height=200, line_color='red', line_width=0.5,
                       fontsize={'xticks':12,'yticks':12 ,'ylabel':14},
                       title='').opts(hooks=[hook])

## Checking missing values (NaN) and filling appropriate values

In [ ]:
met_check.df.index

In [ ]:
datetime_index.values

In [ ]:
datetime_index.values[0] in met_check.df.index.values

In [ ]:
mask = np.logical_not(np.isin(datetime_index.values,met_check.df.index.values))

In [ ]:
datetime_index[mask]